In [122]:
pip install webdriver-manager


In [1]:
from selenium import webdriver
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import re
from selenium.webdriver.common.keys import Keys
driver = webdriver.Chrome(ChromeDriverManager().install())
import time
import getpass
from bs4 import BeautifulSoup
usuario= input("ingrese el usuario para navegaren uss")
contraseña =  getpass.getpass("ingrese la contraseña")
def if_integer(string):
    try: 
        int(string)
        return True
    except ValueError:
        return False

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\rober\.wdm\drivers\chromedriver\win32\103.0.5060.53]
C:\Users\rober\AppData\Local\Temp/ipykernel_6632/4164434000.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


ingrese el usuario para navegaren ussrnerisn@correo.uss.cl
ingrese la contraseña········


In [2]:
#se cargan la pagina de web of science desde selenium
driver.get("http://www.webofscience.com.bdigitaluss.remotexs.co/wos/woscc/summary/3a69e474-ad67-429a-baf5-12e0c77626b7-4057bdc8/relevance/1")
time.sleep(5)

driver.find_element_by_xpath("/html/body/div[4]/div/div/form/div/a").click()
time.sleep(5)
mBox = driver.find_element_by_xpath('//*[@id="i0116"]')
mBox.send_keys(usuario)
time.sleep(2)
driver.find_element_by_xpath('//*[@id="idSIButton9"]').click()
time.sleep(7)
mBox= driver.find_element_by_xpath('//*[@id="i0118"]')
mBox.send_keys(contraseña)
driver.find_element_by_xpath('//*[@id="idSIButton9"]').click()
time.sleep(7)
driver.find_element_by_xpath('//*[@id="idBtn_Back"]').click()
time.sleep(10)
driver.refresh()


C:\Users\rober\AppData\Local\Temp/ipykernel_6632/2281787528.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("/html/body/div[4]/div/div/form/div/a").click()
C:\Users\rober\AppData\Local\Temp/ipykernel_6632/2281787528.py:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  mBox = driver.find_element_by_xpath('//*[@id="i0116"]')
C:\Users\rober\AppData\Local\Temp/ipykernel_6632/2281787528.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="idSIButton9"]').click()
C:\Users\rober\AppData\Local\Temp/ipykernel_6632/2281787528.py:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  mBox= driver.find_element_by_xpath('//*[@id="i0118"]')
C:\Users\rober\

In [3]:
total =int(driver.find_element_by_xpath('/html/body/app-wos/div/div/main/div/div[2]/app-input-route/app-base-summary-component/div/div[2]/app-page-controls[2]/div/form/div/span').text.replace(",",""))
print(total)
datos =[]
driver.maximize_window()
htmlelement= driver.find_element_by_tag_name('html')
for i in range(total):
    htmlelement.send_keys(Keys.END)
    time.sleep(1)
    htmlelement.send_keys(Keys.END)
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight/2)")
    time.sleep(1)
    htmlelement.send_keys(Keys.END)
    time.sleep(1)
    htmlelement.send_keys(Keys.HOME)

    d2 = driver.page_source
    datos.append(d2)
    time.sleep(1)
    driver.find_element_by_xpath("/html/body/app-wos/div/div/main/div/div[2]/app-input-route/app-base-summary-component/div/div[2]/app-page-controls[1]/div/form/div/button[2]").click()
    time.sleep(1)
        


C:\Users\rober\AppData\Local\Temp/ipykernel_6632/1385274439.py:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  total =int(driver.find_element_by_xpath('/html/body/app-wos/div/div/main/div/div[2]/app-input-route/app-base-summary-component/div/div[2]/app-page-controls[2]/div/form/div/span').text.replace(",",""))
C:\Users\rober\AppData\Local\Temp/ipykernel_6632/1385274439.py:5: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  htmlelement= driver.find_element_by_tag_name('html')


36


C:\Users\rober\AppData\Local\Temp/ipykernel_6632/1385274439.py:20: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("/html/body/app-wos/div/div/main/div/div[2]/app-input-route/app-base-summary-component/div/div[2]/app-page-controls[1]/div/form/div/button[2]").click()


In [4]:
print(len(datos))

36


In [5]:
data= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all('a', attrs={'data-ta': 'summary-record-title-link'}):
        data.append(link.getText())

In [6]:
print(len(data))

1790


In [7]:
data2= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all(attrs={'class':'citations ng-star-inserted'}):
        data2.append(link.getText())

In [8]:
for j in range(len(data2)):
    msg = data2[j]
    removeSpaces = re.sub('\\s+', '',msg)
    data2[j] = removeSpaces

In [9]:
print(len(data2))

966


In [10]:
for i in range(len(data2)):
    data2[i] = data2[i].replace("Citations", "")
    data2[i] = data2[i].replace("Citation", "")

In [11]:
tamaño =int(driver.find_element_by_xpath('/html/body/app-wos/div/div/main/div/div[2]/app-input-route/app-base-summary-component/div/div[2]/app-page-controls[2]/div/wos-select/button/span[2]').text)

C:\Users\rober\AppData\Local\Temp/ipykernel_6632/826373608.py:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tamaño =int(driver.find_element_by_xpath('/html/body/app-wos/div/div/main/div/div[2]/app-input-route/app-base-summary-component/div/div[2]/app-page-controls[2]/div/wos-select/button/span[2]').text)


In [12]:
data3= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for k in range(tamaño):
        for link in soup.find_all(attrs={'data-ta': 'SumAuthTa-' + str(k) + '-MainDiv-author-en'}):
            data3.append(link.getText())

In [13]:
print(len(data3))

1788


In [14]:
data4= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all(attrs={'data-ta':'summary-record-pubdate'}):
        data4.append(link.getText())

In [15]:
for i in range(len(data4)):
    data4[i] = data4[i].replace("|","")

In [16]:
helpData4= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all(attrs={'class':'source-info-piece ng-star-inserted'}):
        helpData4.append(link.getText())

In [17]:
for i in range(len(helpData4)):
    helpData4[i] = helpData4[i].replace("|","")

In [18]:
helpData5 = []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all(attrs={'class':'font-size-14 summary-source-title ng-star-inserted'}):
        helpData5.append(link.getText())

In [19]:
print(len(helpData5))

285


In [20]:
data5= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all(attrs={'class':'mat-tooltip-trigger font-size-14 borderLess-button thin-focus summary-source-title-link ng-star-inserted'}):
        data5.append(link.getText())

In [21]:
print(len(data5))

1505


In [22]:
data6= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all(attrs={'class':'link-container ref-count ng-star-inserted'}):
        data6.append(link.getText())

In [23]:
 for i in range(len(data6)):
        data6[i] = data6[i].replace("References ","")
        data6[i] = data6[i].replace("Reference ","")

In [24]:
iterations= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all('app-record',attrs={'ng-star-inserted'}):
        iterations.append(link)

In [25]:
print(len(iterations))

1790


In [29]:
scrappdata = []
scrappslice = []
countdata1 = 0
countdata3 = 0
countdata2 = 0
countdata4 = 0
countdata4help = 0
countdata5 = 0
countdata5help = 0
countit = 0
for i in range(int(len(iterations))):
    if countit == 50:
        countit = 0
    scrappslice = []
    if bool(iterations[i].find_all(attrs={'data-ta':'summary-record-pubdate'})) == True and bool(iterations[i].find_all(attrs={'class':'source-info-piece ng-star-inserted'})) == False and bool(iterations[i].find_all(attrs={'class':'mat-tooltip-trigger font-size-14 borderLess-button thin-focus summary-source-title-link ng-star-inserted'})) == True:
        scrappslice.append(data[countdata1])
        if if_integer(data4[countdata4]) == False:
            dataslice = data4[countdata4].split()
            if len(dataslice) == 2:
                scrappslice.append(dataslice[1])
            else:
                scrappslice.append(dataslice[2])
            scrappslice.append(dataslice[0])    
        else:
            scrappslice.append(data4[countdata4])
            scrappslice.append("-")
        countdata4 += 1
        if bool(iterations[i].find_all(attrs={'data-ta': 'SumAuthTa-' + str(countit) + '-MainDiv-author-en'})) == True:
            scrappslice.append(data3[countdata3])
            countdata3 +=1
        else:
            scrappslice.append("-")
        scrappslice.append(data5[countdata5])
        countdata5 += 1
        if bool(iterations[i].find_all(attrs={'class':'citations ng-star-inserted'})) == True:
            scrappslice.append(data2[countdata2])
            countdata2 += 1
        else:
            scrappslice.append("-")
        scrappslice.append(data6[countdata1])           
    elif bool(iterations[i].find_all(attrs={'data-ta':'summary-record-pubdate'})) == True  and bool(iterations[i].find_all(attrs={'class':'source-info-piece ng-star-inserted'})) == False and bool(iterations[i].find_all(attrs={'class':'font-size-14 summary-source-title ng-star-inserted'})) == True:
        scrappslice.append(data[countdata1])
        if if_integer(data4[countdata4]) == False:
            dataslice = data4[countdata4].split()
            if len(dataslice) == 2:
                scrappslice.append(dataslice[1])
            else:
                scrappslice.append(dataslice[2])
            scrappslice.append(dataslice[0])    
        else:
            scrappslice.append(data4[countdata4])
            scrappslice.append("-")
        countdata4 += 1
        if bool(iterations[i].find_all(attrs={'data-ta': 'SumAuthTa-' + str(countit) + '-MainDiv-author-en'})) == True:
            scrappslice.append(data3[countdata3])
            countdata3 +=1
        else:
            scrappslice.append("-")
        scrappslice.append(helpData5[countdata5help])
        countdata5help +=1
        if bool(iterations[i].find_all(attrs={'class':'citations ng-star-inserted'})) == True:
            scrappslice.append(data2[countdata2])
            countdata2 += 1
        else:
            scrappslice.append("-")
        scrappslice.append(data6[countdata1])
    elif bool(iterations[i].find_all(attrs={'data-ta':'summary-record-pubdate'})) == False and bool(iterations[i].find_all(attrs={'class':'source-info-piece ng-star-inserted'})) == True and bool(iterations[i].find_all(attrs={'class':'mat-tooltip-trigger font-size-14 borderLess-button thin-focus summary-source-title-link ng-star-inserted'})) == True:
        scrappslice.append(data[countdata1])
        dataslice = helpData4[countdata4help].split()
        scrappslice.append(dataslice[1])
        scrappslice.append(dataslice[0])
        countdata4help += 1
        if bool(iterations[i].find_all(attrs={'data-ta': 'SumAuthTa-' + str(countit) + '-MainDiv-author-en'})) == True:
            scrappslice.append(data3[countdata3])
            countdata3 +=1
        else:
            scrappslice.append("-")
        scrappslice.append(data5[countdata5])
        countdata5 += 1
        if bool(iterations[i].find_all(attrs={'class':'citations ng-star-inserted'})) == True:
            scrappslice.append(data2[countdata2])
            countdata2 += 1
        else:
            scrappslice.append("-")
        scrappslice.append(data6[countdata1])
    elif bool(iterations[i].find_all(attrs={'data-ta':'summary-record-pubdate'})) == False and bool(iterations[i].find_all(attrs={'class':'source-info-piece ng-star-inserted'})) == True and bool(iterations[i].find_all(attrs={'class':'font-size-14 summary-source-title ng-star-inserted'})) == True:
        scrappslice.append(data[countdata1])
        dataslice = helpData4[countdata4help].split()
        scrappslice.append(dataslice[1])
        scrappslice.append(dataslice[0])
        countdata4help += 1
        if bool(iterations[i].find_all(attrs={'data-ta': 'SumAuthTa-' + str(countit) + '-MainDiv-author-en'})) == True:
            scrappslice.append(data3[countdata3])
            countdata3 +=1
        else:
            scrappslice.append("-")
        scrappslice.append(helpData5[countdata5help])
        countdata5help += 1
        if bool(iterations[i].find_all(attrs={'class':'citations ng-star-inserted'})) == True:
            scrappslice.append(data2[countdata2])
            countdata2 += 1
        else:
            scrappslice.append("-")
        scrappslice.append(data6[countdata1])
    elif bool(iterations[i].find_all(attrs={'class':'source-info-piece ng-star-inserted'})) == True and bool(iterations[i].find_all(attrs={'data-ta':'summary-record-pubdate'})) == True and bool(iterations[i].find_all(attrs={'class':'mat-tooltip-trigger font-size-14 borderLess-button thin-focus summary-source-title-link ng-star-inserted'})) == True:
        scrappslice.append(data[countdata1])
        if if_integer(data4[countdata4]) == False:
            dataslice = data4[countdata4].split()
            if len(dataslice) == 2:
                scrappslice.append(dataslice[1])
            else:
                scrappslice.append(dataslice[2])
            scrappslice.append(dataslice[0])    
        else:
            scrappslice.append(data4[countdata4])
            scrappslice.append("-")
        countdata4 += 1
        if bool(iterations[i].find_all(attrs={'data-ta': 'SumAuthTa-' + str(countit) + '-MainDiv-author-en'})) == True:
            scrappslice.append(data3[countdata3])
            countdata3 +=1
        else:
            scrappslice.append("-")
        scrappslice.append(data5[countdata5])
        countdata5 += 1
        if bool(iterations[i].find_all(attrs={'class':'citations ng-star-inserted'})) == True:
            scrappslice.append(data2[countdata2])
            countdata2 += 1
        else:
            scrappslice.append("-")
        scrappslice.append(data6[countdata1])
    elif bool(iterations[i].find_all(attrs={'class':'source-info-piece ng-star-inserted'})) == True and bool(iterations[i].find_all(attrs={'data-ta':'summary-record-pubdate'})) == True and bool(iterations[i].find_all(attrs={'class':'font-size-14 summary-source-title ng-star-inserted'})) == True: 
        scrappslice.append(data[countdata1])
        if if_integer(data4[countdata4]) == False:
            dataslice = data4[countdata4].split()
            if len(dataslice) == 2:
                scrappslice.append(dataslice[1])
            else:
                scrappslice.append(dataslice[2])
            scrappslice.append(dataslice[0])    
        else:
            scrappslice.append(data4[countdata4])
            scrappslice.append("-")
        countdata4 += 1
        if bool(iterations[i].find_all(attrs={'data-ta': 'SumAuthTa-' + str(countit) + '-MainDiv-author-en'})) == True:
            scrappslice.append(data3[countdata3])
            countdata3 +=1
        else:
            scrappslice.append("-")
        scrappslice.append(helpData5[countdata5help])
        countdata5help +=1
        if bool(iterations[i].find_all(attrs={'class':'citations ng-star-inserted'})) == True:
            scrappslice.append(data2[countdata2])
            countdata2 += 1
        else:
            scrappslice.append("-")
        scrappslice.append(data6[countdata1])

    countdata1 += 1
    scrappdata.append(scrappslice)
    countit += 1

In [30]:
print(len(scrappdata))

1790


In [32]:
import xlsxwriter
encabezados = [["Titulo","Año","Mes","Autores","Revista","Citas","referencias"]]
scrappdata = encabezados + scrappdata
workbook = xlsxwriter.Workbook('DatosWoS.xlsx')
worksheet = workbook.add_worksheet()
array = scrappdata
row = 0

for col, dataentry in enumerate(array):
    worksheet.write_row(col, row, dataentry)

workbook.close()


In [43]:
hrefparts = []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all('a', attrs={'data-ta': 'summary-record-title-link'}):
        hrefparts.append(link)


In [44]:
datos2 = []
for i in range(len(hrefparts)):
    driver.get('http://www.webofscience.com.bdigitaluss.remotexs.co' + str(hrefparts[i].attrs["href"]))
    time.sleep(2)
    d2 = driver.page_source
    datos2.append(d2)


KeyboardInterrupt: 

In [59]:
print(len(datos2))

642


In [231]:
scrapBook = []
for i in range(len(datos2)):
    Keywords_Plus_list = []
    autors = []
    Research_Areas = []
    scrapBookpart = []
    n = 0
    autorslice = ""
    Keywords_Plus_list_slice = ""
    Research_Areas_slice= ""
    soup =  BeautifulSoup(datos2[i])
    for link in soup.find_all(attrs={'cdxanalyticscategory':'wos-author_keyword_link'}):
        autors.append(link.getText())
    for link in soup.find_all(attrs={'cdxanalyticscategory':'wos-keywords-plus-link'}):
        Keywords_Plus_list.append(link.getText())
    for o in soup.find_all('span',attrs={'class':'value-wrap ng-star-inserted'}):
        n += 1
    for j in range(n):
        for link in soup.find_all('span',attrs={'data-ta':'CategoriesTa-subject-' + str(j)}):
            Research_Areas.append(link.getText())
    if bool(autors) == True:
        for m in range(len(autors)):
            if m != 0:
                autorslice += ";" +autors[m] 
            else:
                autorslice += autors[m] 
            
        scrapBookpart.append(autorslice)
    else:
        scrapBookpart.append("-")
    if bool(Keywords_Plus_list) == True:

        for m in range(len(Keywords_Plus_list)):
            if m != 0:
                Keywords_Plus_list_slice +=  ";" + Keywords_Plus_list[m] 
            else:
                Keywords_Plus_list_slice += Keywords_Plus_list[m] 
        scrapBookpart.append(Keywords_Plus_list_slice)
    else:
        scrapBookpart.append("-")
    if bool(Research_Areas) == True: 
        for m in range(len(Research_Areas)):
            if m != 0:
                Research_Areas_slice += ";" + Research_Areas[m] 
            else:        
                Research_Areas_slice += Research_Areas[m] 
        scrapBookpart.append(Research_Areas_slice) 
    else:
        scrapBookpart.append("-")
    scrapBook.append(scrapBookpart)

In [234]:
import xlsxwriter
encabezados = [["Author Keywords","Keywords Plus","Research Areas"]]
scrapBook = encabezados + scrapBook
workbook = xlsxwriter.Workbook('DatosBookWoS.xlsx')
worksheet = workbook.add_worksheet()
array = scrapBook
row = 0

for col, dataentry in enumerate(array):
    worksheet.write_row(col, row, dataentry)

workbook.close()


In [36]:
hrefs = []
for i in range(len(hrefparts)):
    hrefs.append(str(hrefparts[i].attrs["href"]))
dfhref = pd.DataFrame(hrefs,columns=["href"])
dfhref.to_csv('hrefparts.csv',index=False)

In [74]:
doctypes = []
doctypescon = []
for i in range(len(datos2)):
    n = 0
    doctypes_slice = []
    doctypes = []
    soup =  BeautifulSoup(datos2[i])
    for o in soup.find_all('class',attrs={'class':'value ng-star-inserte'}):
        n += 1

    for link in soup.find_all(attrs={'data-ta':'FullRTa-doctype-' + str(n)}):
        doctypes.append(link.getText())
        for m in range(len(doctypes)):
            if m != 0:
                doctypes_slice += ";" + doctypes[m] 
            else:
                doctypes_slice += doctypes[m]
        doctypescon.append(doctypes)

In [77]:
dfdoctype = pd.DataFrame(doctypescon,columns=["tipo"])
dfdoctype.to_csv('Docktypes.csv',index=False)

In [ ]:
docktype.drop()